<a href="https://colab.research.google.com/github/onlymayank11/Mayank-kptncook-data-challenge/blob/main/KPTNcook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below code will updates the data once a user has listened to an artist. Suppose, user is "33" and he/she listened to "abba" then designated cell in the excel will change from 0 to 1.


In [36]:
# ---  Imported all necesary Libraries  ---
import pandas as pd
from scipy.spatial.distance import cosine   # For finding the reccomandation
import scipy as sp
from scipy import special  
import numpy as np

In [37]:
# Will create a 2D array copy of the csv file.
file_name = 'lastfm-matrix-germany.csv'

#####################################################
import csv
data = []
header = []

with open('lastfm-matrix-germany.csv') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)
    for row in reader:
        data.append(row)
    csvfile.close()

In [38]:
# User information that has to be update.

user_id = 33
artist = "abba"

#------------------------------

user_index = -1
for row in data:
    if row[0] == str(user_id):
        user_index = data.index(row)

artist_index = -1
for col in header[1:]:
    if col == artist:
        artist_index = header.index(col)

if user_index >= 0:
    if artist_index >= 0:
        data[user_index][artist_index] = 1
        print("Updated")
    else:
        print("Artist does not exist in file")
else:
    print("User does not exist in file")

Updated


In [39]:
# Writing the contents of the 2D array back to csv.
# If user hear the new song then this code will run otherwise changes made to the array will not be saved. 
with open(file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(data)
    csvfile.close()

Below code is for Recommending an artist for a specific user. 
The recommendations is on
- Random artists,
- Still unknown artists, or
- Artists that are similar to the ones she already listened to. 


I use the collaborating Filtering technique which is widely used in recommendation system.

I used both User-Based and Item-Based filtering for finding recommendation for the user.



In [40]:
#------------------- Read the file
data = pd.read_csv('lastfm-matrix-germany.csv')

In [41]:
data.head(6).iloc[:,2:8]

,abba,ac/dc,adam green,aerosmith,afi,air
0,0,0,0,0,0,0
1,1,0,1,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0


#Item Based Collaborative Filtering


In [42]:
#Drop the user coloum as it is not important in item based filtering

data_germany = data.drop('user', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


We need a location to store our similarities before we can compute them, so I created a variable named "data_ibs", which is a Pandas Data Frame.


In [43]:
data_ibs = pd.DataFrame(index=data_germany.columns,columns=data_germany.columns)

In [44]:
# Scipy library has a function that allows to do cosine similarities
#cosine similarity divides the sum product of the first and second columns by the product of the square root of the sum of squares of each column.

for i in range(0,len(data_ibs.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(data_ibs.columns)) :
      # Fill in placeholder with cosine similarities
      data_ibs.iloc[i,j] = 1-cosine(data_germany.iloc[:,i],data_germany.iloc[:,j])

Similarity matrix filled out we can look for each items “neighbour” by looping through ‘data_ibs’, sorting each column in descending order, and grabbing the name of each of the top 10 songs.

In [45]:
# Create a placeholder items for closes neighbours to an item
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,11))

for i in range(0,len(data_ibs.columns)):
    data_neighbours.iloc[i,:10] = data_ibs.iloc[0:,i].sort_values(ascending=False)[:10].index
 

#User Based collaborative Filtering

Check which items the user has consumed

For each item the user has consumed, get the top 10 neighbours

Get the consumption record of the user for each neighbour.

Recommending the items with the highest score

In [46]:
data_neighbours.head(8).iloc[:8,1:6] 

,2,3,4,5,6
a perfect circle,tool,dredg,deftones,porcupine tree,nine inch nails
abba,madonna,robbie williams,elvis presley,michael jackson,queen
ac/dc,red hot chili peppers,metallica,iron maiden,the offspring,black sabbath
adam green,the libertines,the strokes,babyshambles,radiohead,franz ferdinand
aerosmith,u2,led zeppelin,metallica,ac/dc,lenny kravitz
afi,funeral for a friend,rise against,fall out boy,anti-flag,sum 41
air,massive attack,goldfrapp,morcheeba,thievery corporation,jamiroquai
alanis morissette,tori amos,alicia keys,red hot chili peppers,kelly clarkson,dido


In [47]:
# Helper function to get similarity scores
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities)

In [48]:
# Create a place holder matrix for similarities, and fill in the user name column
data_sims = pd.DataFrame(index=data.index,columns=data.columns)
data_sims.iloc[:,:1] = data.iloc[:,:1]

In [32]:
#Loop through all rows, skip the user column, and fill with similarity scores
for i in range(0,len(data_sims.index)):
    for j in range(1,len(data_sims.columns)):
        user = data_sims.index[i]
        product = data_sims.columns[j]
 
        if data.iloc[i][j] == 1:
            data_sims.iloc[i][j] = 0
        else:
            product_top_names = data_neighbours.loc[product][1:10]
            product_top_sims = data_ibs.loc[product].sort_values(ascending=False)[1:10]
            user_purchases = data_germany.loc[user,product_top_names]
 
            data_sims.iloc[i][j] = getScore(user_purchases,product_top_sims)

In [33]:
# Get the top songs
data_recommend = pd.DataFrame(index=data_sims.index, columns=['user','1','2','3','4','5','6'])
data_recommend.iloc[0:,0] = data_sims.iloc[:,0]

In [34]:
# Instead of top song scores, we want to see names
for i in range(0,len(data_sims.index)):
    data_recommend.iloc[i,1:] = data_sims.iloc[i,:].sort_values(ascending=False).iloc[1:7,].index.transpose()

In [35]:
data_recommend.iloc[:10,:6]

,user,1,2,3,4,5
0,1,flogging molly,coldplay,aerosmith,the beatles,mando diao
1,33,red hot chili peppers,kings of leon,peter fox,gentleman,flyleaf
2,42,oomph!,lacuna coil,rammstein,schandmaul,sonata arctica
3,51,the subways,the kooks,franz ferdinand,the hives,jack johnson
4,62,jack johnson,incubus,mando diao,the fratellis,oasis
5,75,hoobastank,papa roach,sum 41,blink-182,good charlotte
6,130,alanis morissette,the smashing pumpkins,pearl jam,nickelback,norah jones
7,141,machine head,sonic syndicate,caliban,hatebreed,killswitch engage
8,144,editors,nada surf,the strokes,death cab for cutie,the decemberists
9,150,placebo,the subways,eric clapton,the streets,bob dylan
